In [1]:
import os
import json
import pandas as pd
from PIL import Image
import numpy as np

In [2]:
# Load metadata file
meta_path = 'PKLot/train/_annotations.coco.json'
with open(meta_path) as file:
    meta_train = json.load(file)


In [3]:
# Create DataFrame from the metadata
train = pd.DataFrame(meta_train['images'])
df_meta = pd.DataFrame(meta_train['annotations'])
train = pd.DataFrame.merge(train,df_meta, how='left', left_on='id', right_on='image_id')
train = train.dropna()
train['id_y'] = train['id_y'].values.astype(int)
train['category_id'] = train['category_id'].values.astype(int)
train['area'] = train['area'].values.astype(int)
train['iscrowd'] = train['iscrowd'].values.astype(int)

In [4]:
train_img_folder = 'PKLot/train'
image = []
for img in train['file_name'].unique():
    img_path = os.path.join(train_img_folder, img)
    pixels = np.array(Image.open(img_path))
    image.append(pixels)


In [5]:
images = np.array(image)
print(type(images), images.shape)

<class 'numpy.ndarray'> (8502, 640, 640, 3)


In [6]:
train_small = train[['file_name', 'bbox']].copy()
df_grouped = train_small.groupby('file_name', as_index=True).agg({'bbox': list})
bboxes = df_grouped['bbox'].values.tolist()

In [7]:
import numpy as np

max_boxes = 100
padded_bboxes = np.zeros((len(bboxes), max_boxes, 4))
for i, boxes in enumerate(bboxes):
    padded_bboxes[i, :len(boxes), :] = boxes
padded_bboxes = padded_bboxes.reshape(len(bboxes), -1)  # Form: (n_samples, 4 * max_boxes)
padded_bboxes = padded_bboxes.tolist()


In [8]:
from keras.applications import MobileNetV2
from keras.models import Model
from keras.layers import Dense, Flatten, Input

base_model = MobileNetV2(input_shape=(640, 640, 3), include_top=False, weights='imagenet')
x = Flatten()(base_model.output)
x = Dense(1000, activation='relu')(x)  # High-level feature representation
output = Dense(4 * 100, activation='linear')(x)  # Predictions for all bounding boxes
model = Model(inputs=base_model.input, outputs=output)


2024-11-26 10:30:44.698289: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-26 10:30:44.710600: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-26 10:30:44.714538: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-26 10:30:44.759026: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/tmp/ipykernel_35314/2168762424.py:

ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2] name: 

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
history = model.fit(images, padded_bboxes, batch_size=32, epochs=10, verbose=1)